In [74]:
nigem_asia_countries = [
    'NiGEM NGFS v1.24.2|China',
    'NiGEM NGFS v1.24.2|India',
    'NiGEM NGFS v1.24.2|Japan',
]

geme3_asia_countries = [
    'China - CHN',
    'India - IND',
    'Japan - JPN',
]

# Make a mapping
asia_country_map = dict(zip(nigem_asia_countries, geme3_asia_countries))

In [75]:
import pandas as pd

def extract_nigem_consumption(df, regions, scenario, model, variant='transition'):
    var_base = {
        'coal': 'Quarterly consumption of coal ; MnToe',
        'gas':  'Quarterly consumption of gas ; MnToe',
        'oil':  'Quarterly consumption of oil ; MnToe'
    }
    var_scen = {
        'coal': f'Quarterly consumption of coal ; MnToe({variant})',
        'gas':  f'Quarterly consumption of gas ; MnToe({variant})',
        'oil':  f'Quarterly consumption of oil ; MnToe({variant})'
    }
    years = [col for col in df.columns if col.isdigit()]
    results = {}

    for region in regions:
        results[region] = {}
        for fuel in ['coal', 'gas', 'oil']:
            row_base = df[
                (df['Region'] == region) & 
                (df['Scenario'] == 'Baseline') & 
                (df['Variable'] == var_base[fuel]) &
                (df['Unit'] == 'MnToe') &
                (df['Model'] == model)
            ]
            row_pct = df[
                (df['Region'] == region) &
                (df['Scenario'] == scenario) &
                (df['Variable'] == var_scen[fuel]) &
                (df['Unit'] == '% difference') &
                (df['Model'] == model)
            ]
            if not row_base.empty and not row_pct.empty:
                base_vals = row_base[years].iloc[0].astype(float)
                pct_vals = row_pct[years].iloc[0].astype(float)
                levels = base_vals * (1 + pct_vals / 100)
                pct_change = levels.pct_change() * 100
                df_out = pd.DataFrame({
                    'year': years,
                    'consumption': levels.values,
                    'consumption_pct_change': pct_change.values
                })
                results[region][fuel] = df_out
            else:
                results[region][fuel] = pd.DataFrame()
    return results

# Usage Example:
df_nigem = pd.read_excel('NiGEM_data.xlsx', sheet_name='data')
regions = ['NiGEM NGFS v1.24.2|United States', 'NiGEM NGFS v1.24.2|Asia', 'NiGEM NGFS v1.24.2|United Kingdom', 'NiGEM NGFS v1.24.2|World'] + nigem_asia_countries
nigem_results = extract_nigem_consumption(
    df_nigem, regions, scenario='Net Zero 2050', model='NiGEM NGFS v1.24.2[REMIND-MAgPIE 3.3-4.8]', variant='transition'
)


Exception ignored in: <function ZipFile.__del__ at 0x1063aea20>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/zipfile/__init__.py", line 1940, in __del__
    self.close()
  File "/opt/anaconda3/lib/python3.12/zipfile/__init__.py", line 1957, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [76]:
def extract_geme3_consumption(df, regions, scenario='Baseline'):
    fuels = {
        'coal': 'Coal',
        'gas': 'Gas',
        'crude_oil': 'Crude Oil'
    }
    indicators = ['Production', 'Imports', 'Exports', 'Investments']
    years = [str(year) for year in range(2021, 2031)]
    results = {}
    for region in regions:
        results[region] = {}
        for fuel_key, fuel_name in fuels.items():
            series = {}
            for ind in indicators:
                var_name = f"{ind}|{fuel_name}"
                filtered = df[
                    (df['Region'] == region) &
                    (df['Scenario'] == scenario) &
                    (df['Variable'] == var_name)
                ]
                if not filtered.empty:
                    vals = filtered[years].iloc[0].values
                    series[ind.lower()] = pd.to_numeric(vals)
                else:
                    series[ind.lower()] = pd.Series([0.0]*len(years))
            consumption = (series['production'] +
                           series['imports'] -
                           series['exports'] -
                           series['investments'])
            df_fuel = pd.DataFrame({
                'year': years,
                'consumption': consumption,
                'consumption_pct_change': pd.Series(consumption).pct_change() * 100
            })
            results[region][fuel_key] = df_fuel
    return results

# Usage Example:
df_geme3 = pd.read_excel('GEME3_IIASA_2025_05_02.xlsx', sheet_name='data', usecols=['Region', 'Scenario', 'Variable'] + [str(y) for y in range(2021, 2031)])
geme3_regions = ['USA - USA', 'Asia', 'United Kingdom - GBR', 'World'] + geme3_asia_countries
geme3_results = extract_geme3_consumption(df_geme3, geme3_regions, scenario='HWTP')


In [90]:
def extract_geme3_combined_consumption(df, regions, scenario='Baseline'):
    fuels = {
        'coal': ['Coal', 'Coal Fired', 'CCS Coal'],
        'gas': ['Gas', 'Gas Fired', 'CCS Gas'],
        'crude_oil': ['Crude Oil', 'Oil', 'Oil Fired'] 
    }
    indicators = ['Production', 'Imports', 'Exports', 'Investments']
    years = [str(year) for year in range(2021, 2031)]
    results = {}
    for region in regions:
        results[region] = {}
        for fuel_key, fuel_variants in fuels.items():
            series = {}
            for ind in indicators:
                # Sum across all relevant variants for each indicator
                total = pd.Series([0.0] * len(years), index=years)
                for fuel_name in fuel_variants:
                    var_name = f"{ind}|{fuel_name}"
                    filtered = df[
                        (df['Region'] == region) &
                        (df['Scenario'] == scenario) &
                        (df['Variable'] == var_name)
                    ]
                    if not filtered.empty:
                        vals = pd.to_numeric(filtered[years].iloc[0].values)
                        total += vals
                series[ind.lower()] = total
            consumption = (series['production'] +
                           series['imports'] -
                           series['exports'] -
                           series['investments'])
            df_fuel = pd.DataFrame({
                'year': years,
                'consumption': consumption.values,
                'consumption_pct_change': pd.Series(consumption).pct_change() * 100
            })
            results[region][fuel_key] = df_fuel
    return results

# Usage Example:
# geme3_results = extract_geme3_combined_consumption(df_geme3, geme3_regions, scenario='Baseline')
df_geme3 = pd.read_excel('GEME3_IIASA_2025_05_02.xlsx', sheet_name='data', usecols=['Region', 'Scenario', 'Variable'] + [str(y) for y in range(2021, 2031)])
geme3_regions = ['USA - USA', 'Asia', 'United Kingdom - GBR', 'World'] + geme3_asia_countries
geme3_results = extract_geme3_consumption(df_geme3, geme3_regions, scenario='SWUC')

with pd.ExcelWriter('coal_gas_oil_comparison.xlsx') as writer:
    for region in nigem_results:
        for fuel in ['coal', 'gas', 'oil']:
            nigem_df = nigem_results[region][fuel]
            geme3_region = {
                'NiGEM NGFS v1.24.2|United States': 'USA - USA',
                'NiGEM NGFS v1.24.2|Asia': 'Asia',
                'NiGEM NGFS v1.24.2|United Kingdom': 'United Kingdom - GBR',
                'NiGEM NGFS v1.24.2|World': 'World'
            }[region]
            fuel_geme3_key = fuel if fuel != 'oil' else 'crude_oil'
            geme3_df = geme3_results[geme3_region][fuel_geme3_key]
            if nigem_df.empty or geme3_df.empty or 'year' not in nigem_df.columns or 'year' not in geme3_df.columns:
                print(f"Skipping {region}, {fuel} -- missing data or columns")
                continue
            nigem_df['year'] = nigem_df['year'].astype(int)
            geme3_df['year'] = geme3_df['year'].astype(int)
            comp = nigem_df.merge(geme3_df, on='year', suffixes=('_nigem', '_geme3'))
            comp = comp[comp['year'] >= 2025]  # <--- Only keep year >= 2025
            print("#######",region,"####",fuel,"#######")
            display(comp)
            comp.to_excel(writer, sheet_name=f"{region.split('|')[-1].strip()}_{fuel}", index=False)



In [ ]:
comp

In [91]:
with pd.ExcelWriter('coal_gas_oil_comparison_asia_countries_SUWC.xlsx') as writer:
    for region in nigem_results:
        for fuel in ['coal', 'gas', 'oil']:
            nigem_df = nigem_results[region][fuel]
            # Updated mapping for Asia components
            if region in asia_country_map:
                geme3_region = asia_country_map[region]
            else:
                geme3_region = {
                    'NiGEM NGFS v1.24.2|United States': 'USA - USA',
                    'NiGEM NGFS v1.24.2|United Kingdom': 'United Kingdom - GBR',
                    'NiGEM NGFS v1.24.2|World': 'World'
                }.get(region, None)
            if geme3_region is None:
                print(f"Skipping {region}, {fuel} -- no matching region")
                continue
            fuel_geme3_key = fuel if fuel != 'oil' else 'crude_oil'
            geme3_df = geme3_results[geme3_region][fuel_geme3_key]
            if nigem_df.empty or geme3_df.empty or 'year' not in nigem_df.columns or 'year' not in geme3_df.columns:
                print(f"Skipping {region}, {fuel} -- missing data or columns")
                continue
            nigem_df['year'] = nigem_df['year'].astype(int)
            geme3_df['year'] = geme3_df['year'].astype(int)
            comp = nigem_df.merge(geme3_df, on='year', suffixes=('_nigem', '_geme3'))
            comp = comp[comp['year'] >= 2025]
            print("#######",region,"####",fuel,"#######")
            display(comp)
            comp.to_excel(writer, sheet_name=f"{region.split('|')[-1].strip()}_{fuel}", index=False)


####### NiGEM NGFS v1.24.2|United States #### coal #######


,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,171.973232,-5.879758,12.161471,-13.251963
4,2026,151.654015,-11.815337,9.803087,-19.392262
5,2027,125.589256,-17.186989,6.491367,-33.782414
6,2028,99.524498,-20.753971,3.224569,-50.325266
7,2029,73.459739,-26.189290,0.912767,-71.693354
8,2030,47.394953,-35.481730,0.821897,-9.955441


####### NiGEM NGFS v1.24.2|United States #### gas #######


,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,666.912445,-1.676920,100.590832,-0.577029
4,2026,631.500824,-5.309786,97.832477,-2.742154
5,2027,581.667084,-7.891318,88.169108,-9.877465
6,2028,531.833344,-8.567399,85.560181,-2.959004
7,2029,481.999603,-9.370180,81.112452,-5.198364
8,2030,432.165833,-10.338965,73.874685,-8.923127


####### NiGEM NGFS v1.24.2|United States #### oil #######


...

,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,85.317703,-4.073304,19.045306,-4.051819
4,2026,77.818073,-8.790239,17.416425,-8.552664
5,2027,67.992420,-12.626441,13.881643,-20.295679
6,2028,58.166777,-14.451087,12.641549,-8.933340
7,2029,48.341139,-16.892182,10.670377,-15.592799
8,2030,38.515504,-20.325617,10.722707,0.490421


####### NiGEM NGFS v1.24.2|Japan #### gas #######


,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,90.841953,-0.312429,33.455412,-2.867510
4,2026,87.057247,-4.166254,32.461749,-2.970111
5,2027,81.172573,-6.759545,31.039613,-4.380960
6,2028,75.287899,-7.249584,29.622484,-4.565549
7,2029,69.403225,-7.816228,28.385849,-4.174649
8,2030,63.518536,-8.478985,23.485210,-17.264375


####### NiGEM NGFS v1.24.2|Japan #### oil #######


,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,140.314289,-1.160903,51.344304,-0.551827
4,2026,136.385178,-2.800222,51.173651,-0.332370
5,2027,131.087570,-3.884299,41.768314,-18.379257
6,2028,125.789963,-4.041274,39.924339,-4.414771
7,2029,120.492355,-4.211471,38.425011,-3.755423
8,2030,115.194756,-4.396627,36.852215,-4.093158


In [92]:
for region in nigem_results:
    for fuel in ['coal', 'gas', 'oil']:
        nigem_df = nigem_results[region][fuel]
        # Updated mapping for Asia components
        if region in asia_country_map:
            geme3_region = asia_country_map[region]
        else:
            geme3_region = {
                'NiGEM NGFS v1.24.2|United States': 'USA - USA',
                'NiGEM NGFS v1.24.2|United Kingdom': 'United Kingdom - GBR',
                'NiGEM NGFS v1.24.2|World': 'World'
            }.get(region, None)
        if geme3_region is None:
            print(f"Skipping {region}, {fuel} -- no matching region")
            continue
        fuel_geme3_key = fuel if fuel != 'oil' else 'crude_oil'
        geme3_df = geme3_results[geme3_region][fuel_geme3_key]
        if nigem_df.empty or geme3_df.empty or 'year' not in nigem_df.columns or 'year' not in geme3_df.columns:
            print(f"Skipping {region}, {fuel} -- missing data or columns")
            continue
        nigem_df['year'] = nigem_df['year'].astype(int)
        geme3_df['year'] = geme3_df['year'].astype(int)
        comp = nigem_df.merge(geme3_df, on='year', suffixes=('_nigem', '_geme3'))
        comp = comp[comp['year'] >= 2025]
        print("#######",region,"####",fuel,"#######")
        display(comp)

####### NiGEM NGFS v1.24.2|United States #### coal #######


,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,171.973232,-5.879758,12.161471,-13.251963
4,2026,151.654015,-11.815337,9.803087,-19.392262
5,2027,125.589256,-17.186989,6.491367,-33.782414
6,2028,99.524498,-20.753971,3.224569,-50.325266
7,2029,73.459739,-26.189290,0.912767,-71.693354
8,2030,47.394953,-35.481730,0.821897,-9.955441


####### NiGEM NGFS v1.24.2|United States #### gas #######


,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,666.912445,-1.676920,100.590832,-0.577029
4,2026,631.500824,-5.309786,97.832477,-2.742154
5,2027,581.667084,-7.891318,88.169108,-9.877465
6,2028,531.833344,-8.567399,85.560181,-2.959004
7,2029,481.999603,-9.370180,81.112452,-5.198364
8,2030,432.165833,-10.338965,73.874685,-8.923127


####### NiGEM NGFS v1.24.2|United States #### oil #######


...

,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,85.317703,-4.073304,19.045306,-4.051819
4,2026,77.818073,-8.790239,17.416425,-8.552664
5,2027,67.992420,-12.626441,13.881643,-20.295679
6,2028,58.166777,-14.451087,12.641549,-8.933340
7,2029,48.341139,-16.892182,10.670377,-15.592799
8,2030,38.515504,-20.325617,10.722707,0.490421


####### NiGEM NGFS v1.24.2|Japan #### gas #######


,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,90.841953,-0.312429,33.455412,-2.867510
4,2026,87.057247,-4.166254,32.461749,-2.970111
5,2027,81.172573,-6.759545,31.039613,-4.380960
6,2028,75.287899,-7.249584,29.622484,-4.565549
7,2029,69.403225,-7.816228,28.385849,-4.174649
8,2030,63.518536,-8.478985,23.485210,-17.264375


####### NiGEM NGFS v1.24.2|Japan #### oil #######


,year,consumption_nigem,consumption_pct_change_nigem,consumption_geme3,consumption_pct_change_geme3
3,2025,140.314289,-1.160903,51.344304,-0.551827
4,2026,136.385178,-2.800222,51.173651,-0.332370
5,2027,131.087570,-3.884299,41.768314,-18.379257
6,2028,125.789963,-4.041274,39.924339,-4.414771
7,2029,120.492355,-4.211471,38.425011,-3.755423
8,2030,115.194756,-4.396627,36.852215,-4.093158
